<a href="https://colab.research.google.com/github/Carlageo9/Practica_2_Random_WalksII/blob/main/Practica_2_RandomWalksII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tópico de Industria I - Tarea 2**

In [1]:
# Módulos: importamos las librerías necesarias para ejecutar las funciones.
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import exponweib

import plotly.graph_objects as go
#### Nuevos módulos
import pandas as pd

from scipy.stats import levy_stable

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Actividad 1: Lévy distribution - N Curvas

In [3]:
#Argumentos de
a = 2.89
c = 1.95
#Valor de resolución
resolution = 50
aux_domain = np.linspace(0,3,resolution)

exponweib_pdf = np.array([exponweib.pdf(i, a, c) for i in aux_domain])

fig_exponweib_pdf = go.Figure()

fig_exponweib_pdf.add_trace(go.Scatter(x = aux_domain,
                                       y = exponweib_pdf,
                                       marker = dict(size = 2),
                                       mode = 'lines',
                                       name = 'Weibull',
                                       showlegend = True))